In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from copy import deepcopy

from autowoe import ReportDeco, AutoWoE

### Чтение  выборок

In [ ]:
data = pd.read_csv("./example_data/data_cat.csv")

In [ ]:
train = data.iloc[:14000, :]
test = data.iloc[14000:, :]

### Параметры модели

Для обучения модели рекомендуется указать тип признаков для обучения.
Поэтому создается словарь features_type с ключами: 

 "real" -- вещественный признак
 
 "cat" --  категориальный.
 
 "date"-- ("%Y%d%m", ("m", "d", "wd", "h", "min"))
 
 Для признаков, которые не размечены, типы будут определены автоматом. Такой вариант будет работать, но качество порядочно просядет. __Однако в этот раз воспользуемся этой опцией и дикт features_type создавать не будем__

In [ ]:
# подробно параметры описаны в Example_1
auto_woe = AutoWoE(monotonic=False,
                     max_bin_count=5,
                     oof_woe=True,
                     regularized_refit=True,
                     p_val=0.05,
                     debug=False,
                     verbose=0
        )
auto_woe = ReportDeco(auto_woe)

In [ ]:
autowoe_fit_params = {"train": train, 
                      "target_name": "isFraud",
}

auto_woe.fit(**autowoe_fit_params)

In [ ]:
pred = auto_woe.predict_proba(test)
roc_auc_score(test[autowoe_fit_params["target_name"]], pred)

##### Замечание
ReportDeco - обертка для построения отчета. Она не обязательна для обучения и применения модели, но обязательна для построения отчета (см последнюю ячейку). Для доступа к атрибутам самой модели необходимо обратится к атрибуту auto_woe._ReportDeco__auto_woe декоратора

### Результат работы автотипизатора

In [ ]:
auto_woe.private_features_type

### Формирование отчета

In [ ]:
report_params = {"automl_date_column": "report_month", # колонка с датой в формате params['datetimeFormat']
                 "output_path": "./AUTOWOE_REPORT_3", # папка, куда сгенерится отчет и сложатся нужные файлы
                 "report_name": "___НАЗВАНИЕ ОТЧЕТА___",
                 "report_version_id": 1,
                 "city": "Воронеж",
                 "model_aim": "___ЦЕЛЬ ПОСТРОЕНИЯ МОДЕЛИ___",
                 "model_name": "___НАЗВАНИЕ МОДЕЛИ___",
                 "zakazchik": "___ЗАКАЗЧИК___",
                 "high_level_department": "___ПОДРАЗДЕЛЕНИЕ___",
                 "ds_name": "___РАЗРАБОТЧИК МОДЕЛИ___",
                 "target_descr": "___ОПИСАНИЕ ЦЕЛЕВОГО СОБЫТИЯ___",
                 "non_target_descr": "___ОПИСАНИЕ НЕЦЕЛЕВОГО СОБЫТИЯ___"}

auto_woe.generate_report(report_params)